# 第9章 質問応答
## 9.3 ChatGPTにクイズを答えさせる

In [ ]:
!pip install datasets openai==0.27 tiktoken tqdm

### 9.3.1 OpenAI API

In [ ]:
import os

# 取得したAPIキーに置き換えてください
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
import openai

# ChatGPTに送るメッセージ
messages = [{"role": "user", "content": "日本で一番高い山は何？"}]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)
print(completion["choices"][0]["message"]["content"])

日本で一番高い山は富士山です。


In [ ]:
messages = [
    {"role": "user", "content": "日本で一番高い山は何？"},
    {
        "role": "assistant",
        "content": "日本で一番高い山は富士山です。",
    },
    {
        "role": "user",
        "content": "一つ前の発言をひらがなに変換してください。",
    },
]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
)
print(completion["choices"][0]["message"]["content"])

にほんでいちばんたかいやまはふじさんです。


In [ ]:
messages = [
    {
        "role": "user",
        "content": "数字を1から10まで読み上げてください。",
    }
]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
    max_tokens=1,
)
print(completion["choices"][0]["message"]["content"])

1


In [ ]:
messages = [
    {
        "role": "user",
        "content": "数字を1から10まで読み上げてください。",
    }
]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0,
    stop="5",
)
print(completion["choices"][0]["message"]["content"])

1、2、3、4、


### 9.3.2 効率的なリクエストの送信

In [ ]:
import asyncio
from typing import Awaitable, Callable, TypeVar
from openai.error import OpenAIError

T = TypeVar("T")

async def retry_on_error(
    openai_call: Callable[[], Awaitable[T]],
    max_num_trials: int = 5,
    first_wait_time: int = 10,
) -> Awaitable[T]:
    """OpenAI API使用時にエラーが返ってきた場合に再試行する"""
    for i in range(max_num_trials):
        try:
            # 関数を実行する
            return await openai_call()
        except OpenAIError as e:
            # 試行回数が上限に達したらエラーを送出
            if i == max_num_trials - 1:
                raise
            print(f"エラーを受け取りました：{e}")
            wait_time_seconds = first_wait_time * (2**i)
            print(f"{wait_time_seconds}秒待機します")
            await asyncio.sleep(wait_time_seconds)

In [ ]:
async def _async_batch_run_chatgpt(
    messages_list: list[list[dict[str, str]]],
    temperature: float,
    max_tokens: int | None,
    stop: str | list[str] | None,
) -> list[str]:
    """OpenAI APIに並列してリクエストを送る"""
    # コルーチンオブジェクトをtasksに格納
    tasks = [
        retry_on_error(
            # ラムダ式で無名関数を定義して渡し、
            # retry_on_error関数の内部で呼び出させる
            openai_call=lambda x=ms: openai.ChatCompletion.acreate(
                model="gpt-3.5-turbo",
                messages=x,
                temperature=temperature,
                max_tokens=max_tokens,
                stop=stop,
            )
        )
        for ms in messages_list
    ]
    # tasks内の非同期処理を実行し結果を収集
    completions = await asyncio.gather(*tasks)
    return [
        c["choices"][0]["message"]["content"] for c in completions
    ]

In [ ]:
def batch_run_chatgpt(
    messages_list: list[list[dict[str, str]]],
    temperature: float = 0.0,
    max_tokens: int | None = None,
    stop: str | list[str] | None = None,
) -> list[str]:
    """非同期処理関数を実行するためのラッパー"""
    return asyncio.run(
        _async_batch_run_chatgpt(
            messages_list, temperature, max_tokens, stop
        )
    )

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### 9.3.3 クイズ用のプロンプトの作成

In [ ]:
from abc import ABCMeta, abstractmethod

class PromptMaker(metaclass=ABCMeta):
    """クイズ用プロンプトを作成するための抽象クラス"""

    @abstractmethod
    def run(self, questions: list[str]) -> list[str]:
        """プロンプトの作成（具体的な実装は継承先で行われる）"""
        pass

In [ ]:
class SimplePromptMaker(PromptMaker):
    """単純なクイズ用プロンプトを作成するクラス"""

    def run(self, questions: list[str]) -> list[str]:
        """プロンプトの作成"""
        return [
            "あなたには今からクイズに答えてもらいます。"
            "問題を与えますので、その解答のみを簡潔に出力してください。\n"
            f"問題：{q}\n"
            "解答："
            for q in questions
        ]

In [ ]:
questions = [
    "日本で一番高い山は何？",
    "日本で一番長い川は何？",
    "日本で一番面積の大きい都道府県はどこ？",
    "日本で一番人口の多い都道府県はどこ？",
]
simple_prompt_maker = SimplePromptMaker()
print(simple_prompt_maker.run(questions)[0])

あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。
問題：日本で一番高い山は何？
解答：


In [ ]:
answers = batch_run_chatgpt(
    [
        [{"role": "user", "content": p}]
        for p in simple_prompt_maker.run(questions)
    ],
    temperature=0.0,
)
print(answers)

['富士山', '信濃川', '北海道', '東京都']


### 9.3.4 API使用料金の見積もり

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoding.encode("日本で一番高い山は何？")

[9080, 22656, 16556, 15120, 87217, 45736, 16995, 58911, 15682, 99849, 11571]

In [ ]:
from datasets import load_dataset

quiz_dataset = load_dataset("llm-book/aio", split="validation")

max_answer_length = 0
for answers in quiz_dataset["answers"]:
    for answer in answers:
        answer_length = len(encoding.encode(answer))
        max_answer_length = max(max_answer_length, answer_length)
print(max_answer_length)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/22335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset aio downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___aio/default/0.0.0/ec8ccbb5e0075041edb5a90e7545577f3fbc3926069af06a2b23c1a0ba5a6959. Subsequent calls will reuse this data.
33


In [ ]:
def calculate_prompt_cost(
    prompts: list[str],
    num_output_tokens: int = 40,
    model: str = "gpt-3.5-turbo",
    usd_per_token: float = 0.0015 / 1000,
):
    """
    プロンプトをOpenAI APIに送信した際にかかる費用を見積もる
    """
    # トークナイザの初期化
    encoding = tiktoken.encoding_for_model(model)

    # 入力プロンプトの合計トークン数を算出
    total_num_prompt_tokens = 0
    for prompt in prompts:
        total_num_prompt_tokens += len(encoding.encode(prompt))

    avg_num_prompt_tokens = total_num_prompt_tokens / len(prompts)
    print(
        "入力プロンプトの平均トークン数:"
        f" {int(avg_num_prompt_tokens)}"
    )

    # モデル出力の合計トークン数を見積もる
    total_num_output_tokens = num_output_tokens * len(prompts)

    # 費用の計算
    total_cost = (
        total_num_prompt_tokens + total_num_output_tokens
    ) * usd_per_token
    print(f"合計コスト: {round(total_cost, 3)} USD")

In [ ]:
questions = quiz_dataset["question"]
prompts = simple_prompt_maker.run(questions)
calculate_prompt_cost(prompts)

入力プロンプトの平均トークン数: 107
合計コスト: 0.221 USD


### 9.3.5 クイズデータセットによる評価

In [ ]:
from datasets import Dataset
from tqdm import tqdm

def get_chatgpt_outputs_for_quiz(
    quiz_prompt_maker: PromptMaker,
    quiz_dataset: Dataset,
    batch_size: int,
) -> list[str]:
    """
    クイズ用のプロンプトを使用した際の
    データセットの各問題に対するモデルの解答を集める
    """
    output_answers: list[str] = []
    with tqdm(total=len(quiz_dataset)) as pbar:
        for batch in quiz_dataset.iter(batch_size=batch_size):
            # 入力の準備
            prompts = quiz_prompt_maker.run(batch["question"])
            inputs = [
                [{"role": "user", "content": p}] for p in prompts
            ]

            # APIにリクエストを送信
            answers = batch_run_chatgpt(inputs)

            # モデルの解答を表示
            for question, answer in zip(batch["question"], answers):
                print(f"問題：{question}")
                print(f"解答：{answer}")
                print()

            output_answers += answers
            pbar.update(len(answers))
    return output_answers

In [ ]:
def calculate_quiz_accuracy(
    output_answers: list[str], correct_answers_list: list[list[str]]
) -> float:
    """モデルの解答と正解の解答例から正解率を算出する"""
    num_correct = 0
    for output_answer, answers in zip(
        output_answers, correct_answers_list
    ):
        # モデルの出力が解答例を一つでも含んでいれば正解とみなす
        num_correct += int(any(a in output_answer for a in answers))
    return num_correct / len(output_answers)

In [ ]:
output_answers = get_chatgpt_outputs_for_quiz(
    simple_prompt_maker, quiz_dataset, batch_size=4
)
accuracy = calculate_quiz_accuracy(
    output_answers, [item["answers"] for item in quiz_dataset]
)
print(f"正解率：{accuracy * 100}")

  0%|          | 4/1000 [00:00<02:31,  6.57it/s]

問題：映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?
解答：ジェット団

問題：氷った海に穴を開けて漁をすることから、漢字で「氷の下の魚」と書くタラ科の魚は何?
解答：スケトウダラ

問題：アメリカとロシア間で結ばれた「戦略兵器削減条約」のことをアルファベット5文字の略称で何という?
解答：START

問題：リンゴが木から落ちる様子を見て「万有引力」を発見したといわれる、17世紀のイギリスの科学者は誰?
解答：アイザック・ニュートン



  1%|          | 8/1000 [00:01<02:31,  6.56it/s]

問題：聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受けた貴族文化を何という?
解答：奈良時代

問題：アメリカ・メジャーリーグの2つのリーグといえば、ナショナルリーグと何リーグ?
解答：アメリカンリーグ

問題：池坊、草月流、小原流といえば、どんな分野の三大流派?
解答：華道

問題：英語で「取り巻く」という意味がある、前後左右から音が聞こえる立体音響を何という?
解答：サラウンドサウンド



  1%|          | 12/1000 [00:01<02:20,  7.05it/s]

...

問題：「冬の大三角」を形成する3つの星は、おおいぬ座のシリウス、こいぬ座のプロキオンとオリオン座の何?
解答：ベテルギウス

問題：江戸時代末期・大坂に洗心堂という私塾を開いた儒学者で、1837年に江戸幕府に対して反乱を起こすも鎮圧され自害した人物は誰?
解答：高野長英

問題：『ブエノスアイレス午前零時』で芥川賞を受賞したのは藤沢周ですが、『暗殺の年輪』で直木賞を受賞したのは誰?
解答：村田沙耶香

問題：手紙やハガキを送った際に、料金を支払った証拠として郵便局で押される、日付入りのスタンプのことを何という?
解答：消印



100%|█████████▉| 996/1000 [10:26<00:00,  5.54it/s]

問題：NBAや格闘技などでみられる、選手が対戦相手に対して挑発的な言葉を発して心理的な揺さぶりをかける戦術を何という?
解答：トラッシュトーク

問題：いい加減で役立たずな人のことを、落語によく登場する間抜けな人物にたとえて何という?
解答：「底抜けの馬鹿」という。

問題：パーソン、パンサー、ジャクソンなどの種類がいる、周囲の環境に合わせて体の色を変えることでおなじみの動物といえば何?
解答：カメレオン

問題：小説『古都』や『伊豆の踊子』などの作品を書き、1968年にノーベル文学賞を受賞した作家は誰?
解答：川端康成



100%|██████████| 1000/1000 [10:27<00:00,  1.59it/s]

問題：いわゆる「古典の三大随筆」と呼ばれる3つの作品といえば、『枕草子』、『方丈記』と何?
解答：『徒然草』

問題：正式名称を「新幹線電気軌道総合試験車」という、新幹線の線路や架線の状態を調査するための車両を車体の色から何という?
解答：赤電

問題：明治11年、東京・紀尾井坂で石川県士族の島田一郎らに暗殺された政治家は誰?
解答：伊藤博文

問題：中国を代表するIT企業の総称「BATH」で表される4つの会社といえば、Baidu、Alibaba、Tencentと何?
解答：Huawei

正解率：50.8


### 9.3.6 文脈内学習

In [ ]:
quiz_train_dataset = load_dataset("llm-book/aio", split="train")
num_in_context_examples = 3
in_context_examples = [
    quiz_train_dataset[i] for i in range(num_in_context_examples)
]

for example in in_context_examples:
    print(f'問題：{example["question"]}')
    print(f'解答：{example["answers"][0]}')

問題：「abc 〜the first〜」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう?
解答：26文字
問題：人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう?
解答：骨川
問題：格闘家ボブ・サップの出身国はどこでしょう?
解答：アメリカ


In [ ]:
class InContextPromptMaker(PromptMaker):
    """文脈内学習を用いたクイズ用プロンプトを作成する"""

    def __init__(self, examples: list[tuple[str, str]]):
        self._prompt = (
            "あなたには今からクイズに答えてもらいます。問題を与えますので、"
            "その解答のみを簡潔に出力してください。\n\n"
        )
        for question, answer in examples:
            self._prompt += f"問題：{question}\n解答：{answer}\n\n"

    def run(self, questions: list[str]) -> list[str]:
        """プロンプトの作成"""
        prompts = [
            self._prompt + f"問題：{q}\n解答：" for q in questions
        ]
        return prompts

q_and_a_list = [
    (e["question"], e["answers"][0]) for e in in_context_examples
]
in_context_prompt_maker = InContextPromptMaker(q_and_a_list)
prompt = in_context_prompt_maker.run(["日本で一番高い山は何？"])[0]
print(prompt)

あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。

問題：「abc 〜the first〜」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう?
解答：26文字

問題：人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう?
解答：骨川

問題：格闘家ボブ・サップの出身国はどこでしょう?
解答：アメリカ

問題：日本で一番高い山は何？
解答：


In [ ]:
questions = quiz_dataset["question"]
prompts = in_context_prompt_maker.run(questions)
calculate_prompt_cost(prompts)

入力プロンプトの平均トークン数: 265
合計コスト: 0.458 USD


In [ ]:
in_context_output_answers = get_chatgpt_outputs_for_quiz(
    in_context_prompt_maker, quiz_dataset, batch_size=4
)
in_context_accuracy = calculate_quiz_accuracy(
    in_context_output_answers,
    [item["answers"] for item in quiz_dataset],
)
print(f"正解率：{in_context_accuracy * 100}")

  0%|          | 4/1000 [00:00<02:52,  5.79it/s]

問題：映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?
解答：ジェット団

問題：氷った海に穴を開けて漁をすることから、漢字で「氷の下の魚」と書くタラ科の魚は何?
解答：鱈

問題：アメリカとロシア間で結ばれた「戦略兵器削減条約」のことをアルファベット5文字の略称で何という?
解答：START

問題：リンゴが木から落ちる様子を見て「万有引力」を発見したといわれる、17世紀のイギリスの科学者は誰?
解答：アイザック・ニュートン



  1%|          | 8/1000 [00:01<02:31,  6.56it/s]

問題：聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受けた貴族文化を何という?
解答：奈良時代

問題：アメリカ・メジャーリーグの2つのリーグといえば、ナショナルリーグと何リーグ?
解答：アメリカンリーグ

問題：池坊、草月流、小原流といえば、どんな分野の三大流派?
解答：華道

問題：英語で「取り巻く」という意味がある、前後左右から音が聞こえる立体音響を何という?
解答：サラウンド



  1%|          | 12/1000 [00:01<02:25,  6.78it/s]

...

問題：「冬の大三角」を形成する3つの星は、おおいぬ座のシリウス、こいぬ座のプロキオンとオリオン座の何?
解答：ベテルギウス

問題：江戸時代末期・大坂に洗心堂という私塾を開いた儒学者で、1837年に江戸幕府に対して反乱を起こすも鎮圧され自害した人物は誰?
解答：吉田松陰

問題：『ブエノスアイレス午前零時』で芥川賞を受賞したのは藤沢周ですが、『暗殺の年輪』で直木賞を受賞したのは誰?
解答：村上春樹

問題：手紙やハガキを送った際に、料金を支払った証拠として郵便局で押される、日付入りのスタンプのことを何という?
解答：消印



100%|█████████▉| 996/1000 [11:22<00:01,  2.44it/s]

問題：NBAや格闘技などでみられる、選手が対戦相手に対して挑発的な言葉を発して心理的な揺さぶりをかける戦術を何という?
解答：トラッシュトーク

問題：いい加減で役立たずな人のことを、落語によく登場する間抜けな人物にたとえて何という?
解答：「座敷童子」

問題：パーソン、パンサー、ジャクソンなどの種類がいる、周囲の環境に合わせて体の色を変えることでおなじみの動物といえば何?
解答：カメレオン

問題：小説『古都』や『伊豆の踊子』などの作品を書き、1968年にノーベル文学賞を受賞した作家は誰?
解答：川端康成



100%|██████████| 1000/1000 [11:23<00:00,  1.46it/s]

問題：いわゆる「古典の三大随筆」と呼ばれる3つの作品といえば、『枕草子』、『方丈記』と何?
解答：『徒然草』

問題：正式名称を「新幹線電気軌道総合試験車」という、新幹線の線路や架線の状態を調査するための車両を車体の色から何という?
解答：赤い電車

問題：明治11年、東京・紀尾井坂で石川県士族の島田一郎らに暗殺された政治家は誰?
解答：伊藤博文

問題：中国を代表するIT企業の総称「BATH」で表される4つの会社といえば、Baidu、Alibaba、Tencentと何?
解答：Huawei

正解率：53.1


### 9.3.7 言語モデルの幻覚に注意

In [ ]:
quiz_example = quiz_dataset[87]
print("問題：" + quiz_example["question"])
print("解答：" + str(quiz_example["answers"][0]) + "\n")

問題：姥山貝塚、西之城貝塚、加曽利貝塚などをはじめ、全国にある貝塚の4分の1が位置する都道府県はどこ?
解答：千葉県



In [ ]:
prompt = simple_prompt_maker.run([quiz_example["question"]])[0]
answer = batch_run_chatgpt([[{"role": "user", "content": prompt}]])[0]
print(f"SimplePromptMakerを用いたモデルの解答：{answer}")

SimplePromptMakerを用いたモデルの解答：大分県


In [ ]:
prompt = in_context_prompt_maker.run([quiz_example["question"]])[0]
answer = batch_run_chatgpt([[{"role": "user", "content": prompt}]])[0]
print(f"InContextPromptMakerを用いたモデルの解答：{answer}")

InContextPromptMakerを用いたモデルの解答：大分県
